In [2]:
import pandas as pd
import numpy as np



In [3]:
volunteer = pd.read_csv('./volunteer_opportunities.csv')
# Create a list of redundant column names to drop
to_drop = ["category_desc", "created_date", "locality", "region", "vol_requests"]

# Drop those columns from the dataset
volunteer_subset = volunteer.drop(to_drop, axis=1)

# Print out the head of volunteer_subset
print(volunteer_subset.head())

   opportunity_id  content_id  event_time  \
0            4996       37004           0   
1            5008       37036           0   
2            5016       37143           0   
3            5022       37237           0   
4            5055       37425           0   

                                               title  hits  \
0  Volunteers Needed For Rise Up & Stay Put! Home...   737   
1                                       Web designer    22   
2      Urban Adventures - Ice Skating at Lasker Rink    62   
3  Fight global hunger and support women farmers ...    14   
4                                      Stop 'N' Swap    31   

                                             summary is_priority  category_id  \
0  Building on successful events last summer and ...         NaN          NaN   
1             Build a website for an Afghan business         NaN          1.0   
2  Please join us and the students from Mott Hall...         NaN          1.0   
3  The Oxfam Action Corps is a g

In [4]:
# Import pandas and wine dataset
import pandas as pd
wine = pd.read_csv('./wine_types.csv')

# Print out the Pearson correlation coefficients for each pair of features in the wine dataset
corr_matrix = wine.corr(method='pearson')
print(corr_matrix)

# Drop any columns from wine that have a correlation coefficient above 0.75 with at least two other columns
cols_to_drop = []
for col in corr_matrix.columns:
    if ((corr_matrix[col].abs() > 0.75).sum() > 2):
        cols_to_drop.append(col)
        
wine = wine.drop(cols_to_drop, axis=1)
print(wine.head())

                                  Type   Alcohol  Malic acid       Ash  \
Type                          1.000000 -0.328222    0.437776 -0.049643   
Alcohol                      -0.328222  1.000000    0.094397  0.211545   
Malic acid                    0.437776  0.094397    1.000000  0.164045   
Ash                          -0.049643  0.211545    0.164045  1.000000   
Alcalinity of ash             0.517859 -0.310235    0.288500  0.443367   
Magnesium                    -0.209179  0.270798   -0.054575  0.286587   
Total phenols                -0.719163  0.289101   -0.335167  0.128980   
Flavanoids                   -0.847498  0.236815   -0.411007  0.115077   
Nonflavanoid phenols          0.489109 -0.155929    0.292977  0.186230   
Proanthocyanins              -0.499130  0.136698   -0.220746  0.009652   
Color intensity               0.265668  0.546364    0.248985  0.258887   
Hue                          -0.617369 -0.071747   -0.561296 -0.074667   
OD280/OD315 of diluted wines -0.788230

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
volunteer = volunteer.dropna(subset=['category_desc'])

# Take the title text
title_text = volunteer["title"]

# Create the vectorizer method
tfidf_vec = TfidfVectorizer()

# Transform the text into tf-idf vectors

text_tfidf = tfidf_vec.fit_transform(title_text)

In [9]:
vocab = {v: k for k, v in tfidf_vec.vocabulary_.items()}

In [7]:
# Add in the rest of the arguments
def return_weights(vocab, original_vocab, vector, vector_index, top_n):
    zipped = dict(zip(vector[vector_index].indices, vector[vector_index].data))
    
    # Transform that zipped dict into a series
    zipped_series = pd.Series({vocab[i]:zipped[i] for i in vector[vector_index].indices})
    
    # Sort the series to pull out the top n weighted words
    zipped_index = zipped_series.sort_values(ascending=False)[:top_n].index
    return [original_vocab[i] for i in zipped_index]

# Print out the weighted words
print(return_weights(vocab, tfidf_vec.vocabulary_, text_tfidf, 8, 3))

[189, 942, 466]


In [11]:
def words_to_filter(vocab, original_vocab, vector, top_n):
    filter_list = []
    for i in range(0, vector.shape[0]):
    
        # Call the return_weights function and extend filter_list
        filtered = return_weights(vocab, original_vocab, vector, i, top_n)
        filter_list.extend(filtered)
        
    # Return the list in a set, so we don't get duplicate word indices
    return set(filter_list)

# Call the function to get the list of word indices
filtered_words = words_to_filter(vocab, tfidf_vec.vocabulary_, text_tfidf, 3)

# Filter the columns in text_tfidf to only those in filtered_words
filtered_text = text_tfidf[:, list(filtered_words)]
filtered_text


<617x1007 sparse matrix of type '<class 'numpy.float64'>'
	with 2946 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

# Split the dataset according to the class distribution of category_desc
X_train, X_test, y_train, y_test = train_test_split(filtered_text.toarray(), volunteer["category_desc"], stratify=volunteer["category_desc"], random_state=42)

# Fit the model to the training data
nb.fit(X_train, y_train)

# Print out the model's accuracy

print(nb.score(X_test, y_test))

0.5161290322580645


In [17]:

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import pandas as pd

# load the wine dataset
wine = pd.read_csv("./wine_types.csv")


# Instantiate a PCA object
pca = PCA()

# Define the features and labels from the wine dataset
X = wine.drop("Type", axis=1)
y = wine["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Apply PCA to the wine dataset X vector
pca_X_train = pca.fit_transform(X_train)
pca_X_test = pca.transform(X_test)

# Look at the percentage of variance explained by each component
print(pca.explained_variance_ratio_)
# ```

[9.97795009e-01 2.02071827e-03 9.88350594e-05 5.66222566e-05
 1.26161135e-05 8.93235789e-06 3.13856866e-06 1.57406401e-06
 1.15918860e-06 7.49332354e-07 3.70332305e-07 1.94185373e-07
 8.08440051e-08]


In [18]:
# Fit knn to the training data
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(pca_X_train, y_train)

# Score knn on the test data and print it out
print(knn.score(pca_X_test, y_test))

0.7777777777777778
